# Local Text-to-Image Generation using older Nvidia GPU and Gradio Interface

- This notebook showcases the setup of a GPU-accelerated text-to-image generator.

- To enhance the user experience, a Gradio interface is implemented for controlling the image generation process.

- The text-to-image generator operates on a local GPU and a locally stored diffusion model.


In [ ]:
# optional: remove warnings
import warnings
warnings.filterwarnings('ignore')

## 1. Create a Project Folder

On your system, create a separate folder for the project. 

## 2. Load the Safetensors Model File

Download the safetensor model file: https://huggingface.co/dreamlike-art/dreamlike-diffusion-1.0/blob/main/dreamlike-diffusion-1.0.safetensors

and move the file into the project folder along with the Jupyter notebook file. 

## 3. Anaconda Navigator

**Environments**: Create a new environment, providing a name and selecting the Python package. 

**Home**: Install JupyterLab and launch it. 

## 4. JupyterLab

Navigate to the project folder. Open the notebook.

Verify that you are working in the newly created environment by running:

In [ ]:
!conda env list

## 5. Connecting CUDA

In the Anaconda Navigator, go to the **Terminal** of the current environment and run: 

## 6. Verifying CUDA-enabled GPU

Check if CUDA is available by running:

In [ ]:
import torch
torch.cuda.is_available()

## 7. Install Gradio, Diffusers, Transformers, Safetensors and OmegaConf

Again, use the **Terminal** of the Anaconda environment for installation:

## 8. Import Libraries and Modules

In [ ]:
import gradio as gr
from diffusers import StableDiffusionPipeline
from diffusers.schedulers.scheduling_dpmsolver_multistep import DPMSolverMultistepScheduler
import safetensors

## 9. Run the Code for Image Generation

In [ ]:

def generate_image(prompt, x_dimension, y_dimension, seed_enabled, seed, enable_inference_steps, num_inference_steps):
   
    # Check if all dimensions are provided
    if prompt and x_dimension and y_dimension:
        
        # Initialize generator with a default value
        generator = torch.Generator("cuda")

        # Define the seed for image generation if seed is enabled
        if seed_enabled:
            seed = int(seed)
            generator.manual_seed(seed)
        else:
            # If seed is not enabled, reset the generator
            generator.manual_seed(torch.seed())
        
        # Create diffusion pipeline, load safetensors model file from project folder (from subfolders: "./"dreamlike-diffusion-1.0.safetensors")
        pipe = StableDiffusionPipeline.from_single_file("dreamlike-diffusion-1.0.safetensors", torch_dtype=torch.float32, use_safetensors=True)
        pipe = pipe.to("cuda")

        # Apply inference steps if enabled
        if enable_inference_steps:
            pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config, use_karras_sigmas=True) 
            image = pipe(prompt, generator=generator, num_inference_steps=num_inference_steps).images[0]
        else:
            image = pipe(prompt, generator=generator).images[0]

        # Resize the image based on the provided dimensions
        dimensions = (int(x_dimension), int(y_dimension))      
        image = image.resize(dimensions)
        
        # After generating the image, clear GPU memory
        torch.cuda.empty_cache()
        
        # Return generated and resized image
        return image

      
# Options for the radio buttons
dimensions = ["512", "640", "768", "1024"]

# Build the Gradio interface
iface_generate_image = gr.Interface(
    fn=generate_image,
    title="Text-to-Image Generator", 
    inputs=[
        gr.Textbox(label="Text Prompt", placeholder="Type your prompt here..."),
        gr.Radio(label="X Dimension", choices=dimensions),
        gr.Radio(label="Y Dimension", choices=dimensions),
        gr.Checkbox(label="Enable Seed"),
        gr.Number(label="Set Seed Number"),
        gr.Checkbox(label="Enable Inference Steps"),
        gr.Slider(label="Set Inference Steps"),
    ],
    outputs=gr.Image(label="Generated Image")
    #allow_flagging="never"
)

# start interface
iface_generate_image.launch()


**Use the following code to completely free up the GPU:**

In [ ]:
# unload GPU
with torch.no_grad(): 
    torch.cuda.empty_cache()